In [1]:
import os
import json
import xml.etree.ElementTree as ET
from datetime import datetime

import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import pystac
import sys
from shapely.geometry import mapping, box
from constants import DEFAULT_START_DATE, DEFAULT_END_DATE,raster_lulc_id,raster_lulc_description,raster_lulc_title,swb_vector_id,swb_vector_title,swb_vector_description,id_main,title_main,description_main,base_url,data_url




/home/vishnu/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [ ]:

base_dir="data/"
qml_path="data/style_file.qml"
vector_qml_file = "data/swb_style.qml"

raster_filename="saraikela-kharsawan_gobindpur_2023-07-01_2024-06-30_LULCmap_10m.tif"
vector_filename="swb2_saraikela-kharsawan_gobindpur.geojson"

corestack_dir = os.path.join(base_dir, "Corestack Catalogs")
gobindpur_dir = os.path.join(corestack_dir, "gobindpur")
raster_dir = os.path.join(gobindpur_dir, "raster")
vector_dir = os.path.join(gobindpur_dir, "vector")

os.makedirs(raster_dir, exist_ok=True)
os.makedirs(vector_dir, exist_ok=True)


raster_filename = raster_filename
vector_filename = vector_filename
raster_path = os.path.join(base_dir, raster_filename)
vector_path = os.path.join(base_dir, vector_filename)

raster_thumb = os.path.join(raster_dir, "thumbnail.png")
vector_thumb = os.path.join(vector_dir, "thumbnail.png")

raster_style_file = os.path.join(base_dir, "style_file.qml")
vector_style_file = os.path.join(base_dir, "swb_style.qml")


In [3]:
def extract_raster_dates_from_filename(filename):
    try:
        print(filename)
        parts = filename.split('_')
        start_date = datetime.strptime(parts[2], "%Y-%m-%d")
        end_date = datetime.strptime(parts[3], "%Y-%m-%d")
        print(start_date)
        print(end_date)
    except Exception as e:
        raise ValueError(f"Failed to extract raster dates from filename '{filename}': {e}")
        
    return start_date, end_date    

In [4]:
extract_raster_dates_from_filename(raster_filename)

saraikela-kharsawan_gobindpur_2023-07-01_2024-06-30_LULCmap_10m.tif
2023-07-01 00:00:00
2024-06-30 00:00:00


(datetime.datetime(2023, 7, 1, 0, 0), datetime.datetime(2024, 6, 30, 0, 0))

In [5]:


def parse_qml_classes(qml_path):
    tree = ET.parse(qml_path)
    root = tree.getroot()
    classes = []

    for entry in root.findall(".//paletteEntry"):
        class_info = {}
        for attr_key, attr_value in entry.attrib.items():
            if attr_key == "value":
                try:
                    class_info[attr_key] = int(attr_value)
                except ValueError:
                    class_info[attr_key] = attr_value
            else:
                class_info[attr_key] = attr_value
        classes.append(class_info)
    return classes

In [6]:


def rgba_string_to_hex(rgba_string):
    
    try:
        parts = rgba_string.split(',')
        if len(parts) >= 3:
            r, g, b = int(parts[0]), int(parts[1]), int(parts[2])
            return "#{:02X}{:02X}{:02X}".format(r, g, b)
    except:
        pass
    return None

def parse_simpleline_style(qml_path):
    tree = ET.parse(qml_path)
    root = tree.getroot()

    # Find the <layer> tag with class="SimpleLine"
    simple_line_layer = None
    for layer in root.findall(".//layer"):
        if layer.attrib.get("class") == "SimpleLine":
            simple_line_layer = layer
            break

    if simple_line_layer is None:
        print("No <layer class='SimpleLine'> found.")
        return {}

    style_data = {}
    for option in simple_line_layer.findall(".//Option"):
        name = option.attrib.get("name")
        value = option.attrib.get("value")
        if name and value:
            style_data[name] = value

    # Convert color if available
    if "line_color" in style_data:
        style_data["line_color_hex"] = rgba_string_to_hex(style_data["line_color"])

    return style_data

 
style = parse_simpleline_style(vector_qml_file)

if not style:
    print("No style properties found.")
else:
    print("Extracted Style Properties:\n")
    for key, value in style.items():
        print(f"{key}: {value}")


Extracted Style Properties:

align_dash_pattern: 0
capstyle: square
customdash: 5;2
customdash_map_unit_scale: 3x:0,0,0,0,0,0
customdash_unit: MM
dash_pattern_offset: 0
dash_pattern_offset_map_unit_scale: 3x:0,0,0,0,0,0
dash_pattern_offset_unit: MM
draw_inside_polygon: 0
joinstyle: bevel
line_color: 225,89,137,255,rgb:0.88235294117647056,0.34901960784313724,0.53725490196078429,1
line_style: solid
line_width: 0.6
line_width_unit: MM
offset: 0
offset_map_unit_scale: 3x:0,0,0,0,0,0
offset_unit: MM
ring_filter: 0
trim_distance_end: 0
trim_distance_end_map_unit_scale: 3x:0,0,0,0,0,0
trim_distance_end_unit: MM
trim_distance_start: 0
trim_distance_start_map_unit_scale: 3x:0,0,0,0,0,0
trim_distance_start_unit: MM
tweak_dash_pattern_on_corners: 0
use_custom_dash: 0
width_map_unit_scale: 3x:0,0,0,0,0,0
type: collection
line_color_hex: #E15989


In [7]:
def generate_raster_thumbnail(tif_path, out_path):
    with rasterio.open(tif_path) as src:
        arr = src.read(1)
    plt.figure(figsize=(3, 3))
    plt.imshow(arr, cmap="tab20")
    plt.axis('off')
    plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
    plt.close()

def generate_vector_thumbnail(vector_path, out_path):
    gdf = gpd.read_file(vector_path)
    if gdf.crs is None or gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)
    fig, ax = plt.subplots(figsize=(3, 3))
    fig.patch.set_facecolor("white")
    ax.set_facecolor("white")
    gdf.plot(ax=ax, color="lightblue", edgecolor="blue", linewidth=0.5)
    ax.axis('off')
    plt.savefig(out_path, dpi=150, bbox_inches='tight', pad_inches=0, facecolor=fig.get_facecolor())
    plt.close()


In [8]:
def generate_legend_image(legend_data, output_path):
    import matplotlib.pyplot as plt
    import matplotlib.patches as mpatches

    labels = [entry.get("label", str(entry["value"])) for entry in legend_data]
    colors = [entry["color"] for entry in legend_data]

    fig_height = max(1, len(labels) * 0.4)
    fig, ax = plt.subplots(figsize=(3, fig_height))
    ax.axis("off")

    handles = [mpatches.Patch(color=color, label=label) for color, label in zip(colors, labels)]
    ax.legend(handles=handles, loc="center left", frameon=False)

    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()


In [9]:
def create_raster_item():
    try:
        start_date, end_date = extract_raster_dates_from_filename(raster_filename)
    except ValueError as e:
        raise RuntimeError(f"Raster item creation failed")
    
    with rasterio.open(raster_path) as src:
        bounds = src.bounds
        geom = mapping(box(*bounds))
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]

    generate_raster_thumbnail(raster_path, raster_thumb)
    style_info = parse_qml_classes(raster_style_file)

   
    style_json_path = os.path.join(raster_dir, "legend.json")
    with open(style_json_path, "w") as f:
        json.dump(style_info, f, indent=2)

    
    legend_img_path = os.path.join(raster_dir, "legend.png")
    generate_legend_image(style_info, legend_img_path)

    

    item = pystac.Item(
        id=raster_lulc_id,
        geometry=geom,
        bbox=bbox,
        datetime=start_date,
        start_datetime= start_date,
        end_datetime= end_date,
        properties={
            "title":"raster_lulc_title",
            "description": "raster_lulc_description",
            "lulc:classes": "style_info",
            
        }
    )
    

    item.add_asset("data", pystac.Asset(
        href=f"{data_url}/{raster_filename}",
        media_type=pystac.MediaType.GEOTIFF,
        roles=["data"],
        title="Raster Layer"
    ))
    item.add_asset("thumbnail", pystac.Asset(
        href=f"{base_url}/raster/thumbnail.png",
        media_type=pystac.MediaType.PNG,
        roles=["thumbnail"],
        title="Raster Thumbnail"
    ))
    item.add_asset("legend_image", pystac.Asset(
        href=f"{base_url}/raster/legend.png",
        media_type=pystac.MediaType.PNG,
        roles=["thumbnail", "metadata"],
        title="Legend Image"
    ))
    item.add_asset("legend", pystac.Asset(
        href=f"{base_url}/raster/legend.json",
        media_type=pystac.MediaType.JSON,
        roles=["metadata"],
        title="Legend JSON"
    ))


    item.set_self_href(os.path.join(raster_dir, "item.json"))
    item.save_object()
    return item


In [10]:
print(base_url)

https://raw.githubusercontent.com/vishnus4059/STAC_common/master/data/Corestack%20Catalogs/gobindpur


In [11]:
raster_item=create_raster_item()

saraikela-kharsawan_gobindpur_2023-07-01_2024-06-30_LULCmap_10m.tif
2023-07-01 00:00:00
2024-06-30 00:00:00


In [12]:
def create_vector_item():
    start_date = DEFAULT_START_DATE
    end_date = DEFAULT_END_DATE

    gdf = gpd.read_file(vector_path)
    geom = mapping(gdf.union_all())
    bounds = gdf.total_bounds
    bbox = [float(b) for b in bounds]

    generate_vector_thumbnail(vector_path, vector_thumb)
    style_info_vector = parse_simpleline_style(vector_style_file)
    style_json_path = os.path.join(vector_dir, "style.json")
    with open(style_json_path, "w") as f:
        json.dump(style_info_vector, f, indent=2)
    
    

    item = pystac.Item(
        id=swb_vector_id,
        geometry=geom,
        bbox=bbox,
        datetime=start_date,
        start_datetime= start_date,
        end_datetime= end_date,
        properties={
            "title": "swb_vector_title",
            "description": "swb_vector_description",
            "style": "style_info_vector",
            
        }
    )


    

    item.add_asset("data", pystac.Asset(
        href=f"{data_url}/{vector_filename}",
        media_type=pystac.MediaType.GEOJSON,
        roles=["data"],
        title="Vector Layer"
    ))
    item.add_asset("thumbnail", pystac.Asset(
        href=f"{base_url}/vector/thumbnail.png",
        media_type=pystac.MediaType.PNG,
        roles=["thumbnail"],
        title="Vector Thumbnail"
    ))
    item.add_asset("style", pystac.Asset(
        href=f"{base_url}/vector/swb_style_thumbnail.json",
        media_type=pystac.MediaType.PNG,
        roles=["style"],
        title="Vector style thumbnail"
    ))
    item.set_self_href(os.path.join(vector_dir, "item.json"))
    item.save_object()
    return item


In [14]:

vector_item = create_vector_item()


DataSourceError: data/data/swb2_saraikela-kharsawan_gobindpur.geojson: No such file or directory

In [ ]:

gobindpur_catalog = pystac.Catalog(
    id=id_main,
    title=title_main,
    description=description_main
)
gobindpur_catalog.add_item(create_raster_item())
gobindpur_catalog.add_item(create_vector_item())
gobindpur_catalog.set_self_href(os.path.join(gobindpur_dir, "catalog.json"))

corestack_catalog = pystac.Catalog(
    id="corestack",
    title="CorestackCatalogs",
    description="Root catalog containing all subcatalogs"
)
corestack_catalog.add_child(gobindpur_catalog)
corestack_catalog.set_self_href(os.path.join(corestack_dir, "catalog.json"))
corestack_catalog.normalize_and_save(corestack_dir, catalog_type=pystac.CatalogType.SELF_CONTAINED)

print(f" Root STAC Catalog created at: {os.path.join(corestack_dir, 'catalog.json')}")


saraikela-kharsawan_gobindpur_2023-07-01_2024-06-30_LULCmap_10m.tif
2023-07-01 00:00:00
2024-06-30 00:00:00
2023-07-01T00:00:00
 Root STAC Catalog created at: data/Corestack Catalogs/catalog.json
